In [1]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr()

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df['High Correlation Count'] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_pearson_correlation_with_label.csv",
        index_label="Packet Interval",
    )

process_attack_data("brute-ssh", "./csvs/")

Correlation table for attack type: brute-ssh
       Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                   0.044         0.016          0.157              0.469        0.174        0.154                  0.032                   0.032              0.043                        0.043                       0
50                   0.057         0.027          0.365              0.524        0.212        0.340                  0.167                   0.166              0.055                        0.055                       0
100                  0.061         0.034          0.523              0.562        0.295        0.484                  0.211                   0.210              0.059                        0.059                       0
250                  0.063         0.048          0.735              0.653 

In [2]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr(method="spearman")

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df["High Correlation Count"] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_spearman_correlation_with_label.csv",
        index_label="Packet Interval",
    )


process_attack_data("brute-ssh", "./csvs/")

Correlation table for attack type: brute-ssh
       Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                   0.042         0.221          0.160              0.353        0.431        0.194                  0.099                   0.099              0.046                        0.046                       0
50                   0.171         0.010          0.086              0.319        0.444        0.008                  0.022                   0.021              0.167                        0.167                       0
100                  0.293         0.156          0.289              0.342        0.437        0.188                  0.015                   0.014              0.283                        0.283                       0
250                  0.419         0.447          0.503              0.452 